In [ ]:
# import splitfolders

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# input_folders = 'F:/sleep monitoring project/Dataset reduce size/Thermal_size224/Thermal_Light_On_Resized/Thermal_Light_On_WithBlanket'
# out_folders = 'F:/sleep monitoring project/Dataset reduce size/Thermal_size224/Thermal_Light_On_Resized/senghour_TV/Thermal_Light_On_WithBlanket'
# splitfolders.ratio(input_folders, out_folders, seed=42, ratio=(0.8, 0.2), group_prefix=None)  # Adjust the ratio here

In [ ]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from keras_preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from keras_preprocessing import image
from tensorflow.keras.models import Sequential, Model
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import tensorflow as tf

In [ ]:
import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
# Check if GPU is available and being used

len(tf.config.list_physical_devices('GPU'))>0


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15408732405113674990
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14328594432
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16105484928121756858
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


True

In [ ]:
train_data = '/content/drive/MyDrive/sleep monitoring project/TM_Lf/Thermal_Light_Off_WithBlanket/train'
val_data = '/content/drive/MyDrive/sleep monitoring project/TM_Lf/Thermal_Light_Off_WithBlanket/val'
test_data = '/content/drive/MyDrive/sleep monitoring project/TM_Lf/Thermal_Light_Off_WithBlanket/Test Thermal_Light_off_WithBlanket'

In [ ]:
img_height, img_width = (224, 224)
batch_size = 128

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    shear_range=0.2,
    zoom_range=0.2
)


In [ ]:
train_datagenerator = train_datagen.flow_from_directory(
    train_data,
    target_size= (img_height, img_width),
    batch_size= batch_size,
    class_mode= "categorical"
)
val_datagenerator = train_datagen.flow_from_directory(
    val_data,
    target_size= (img_height, img_width),
    batch_size= batch_size,
    class_mode= "categorical"
)

test_generate = ImageDataGenerator(rescale=1.0 / 255.0)

Found 51630 images belonging to 5 classes.
Found 12906 images belonging to 5 classes.


In [ ]:
test_datagenerator = test_generate.flow_from_directory(
    test_data,
    target_size= (img_height, img_width),
    batch_size= 128,
    class_mode= "categorical"
)

Found 32772 images belonging to 5 classes.


In [ ]:
x, y = test_datagenerator.next()
x.shape

(128, 224, 224, 3)

*Pre*-Train VGG16 model

In [ ]:
# Define our pre-trained model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras import layers

pre_trained_model = VGG16(include_top=False, weights="imagenet")

# Flatten the output layer to one dimension
x = GlobalAveragePooling2D()(pre_trained_model.output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = Dense(1024, activation='relu')(x)
# Add a final sigmoid layer for classification
predictions = layers.Dense(train_datagenerator.num_classes, activation='softmax')(x)
# Form our model
VGG16_model = Model(inputs=pre_trained_model.input, outputs= predictions)
for layer in pre_trained_model.layers:
    layer.trainable = False
# Compile the model with Adam optimizer and categorical cross-entropy loss.
VGG16_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define the early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with the defined early stopping callback
history = VGG16_model.fit(train_datagenerator, validation_data=val_datagenerator, epochs=50, callbacks=[early_stopping], verbose=1)

Epoch 1/50
117/404 [=======>......................] - ETA: 3:28:39 - loss: 1.0938 - accuracy: 0.6491

In [ ]:
# Plotting Training
plt.plot(history.history['loss'], '-', color='red', label='Training Loss')
plt.plot(history.history['val_loss'], '-', color='green', label='Validation Loss')
plt.plot(history.history['accuracy'], '-', color='black', label='Train Accuracy')
plt.plot(history.history['val_accuracy'], '-', color='Blue', label='Validation Accuracy')
plt.title('Average Training and Validation Set Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
train_loss, train_acc = VGG19_model.evaluate(train_datagenerator, verbose=2)
val_loss, val_acc = VGG19_model.evaluate(val_datagenerator, verbose=2)
test_loss, test_acc = VGG19_model.evaluate(test_datagenerator, verbose=2)


print(f"Training Accuracy: {train_acc} and Training Loss: {train_loss}")
print(f"Validation Accuracy: {val_acc} and Validation Loss: {val_loss}")
print(f"Test Accuracy: {test_acc} and Test Loss: {test_loss}")

In [ ]:
filename= test_datagenerator.filenames
nb_sample = len(test_datagenerator)
y_prob = []
y_act = []
test_datagenerator.reset()

for _ in range(nb_sample):
    X_test, Y_test = test_datagenerator.next()
    y_prob.append(VGG19_model.predict(X_test))
    y_act.append(Y_test)

VGG16_predict_class = [list(train_datagenerator.class_indices.keys())[i.argmax()] for i in y_prob]
VGG16_actual_classes = [list(train_datagenerator.class_indices.keys())[i.argmax()] for i in y_act]

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
classes = ['Left Log', 'Prone Left', 'Prone Right', 'Right Log', 'Supine_Thermal']
print("Test Result:================================================")
print(f"Accuracy Score: {accuracy_score(VGG16_actual_classes, VGG16_predict_class) * 100:.2f}%")
print("_______________________________________________")
print(f"CLASSIFICATION REPORT:\n{classification_report(VGG16_actual_classes, VGG16_predict_class, target_names=classes)}")
print("_______________________________________________")
print(f"Confusion Matrix: \n{confusion_matrix(VGG16_actual_classes, VGG16_predict_class)}")

In [ ]:
class_labels = ['Left Log', 'Prone Left', 'Prone Right', 'Right Log', 'Supine']  # Modify the labels based on your actual class names
VGG16_cm = confusion_matrix(VGG16_actual_classes, VGG16_predict_class)



# Plot confusion matrix
plt.figure(figsize=(8, 6))
plt.imshow(VGG16_cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix of Testing Data')
plt.colorbar()
tick_marks = np.arange(len(class_labels))
plt.xticks(tick_marks, class_labels, rotation=45)
plt.yticks(tick_marks, class_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')

# Add labels to each cell cmap='Blues', annot=True, fmt='d'
for i in range(len(class_labels)):
    for j in range(len(class_labels)):
        plt.text(j, i, str(VGG16_cm[i, j]), ha='center', va='center', color='orange')

plt.show()

In [5]:
from PIL import Image
import os

def resize_and_rotate_images(folder_path):
    for root, _, files in os.walk(folder_path):
        for filename in files:
            if filename.endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(root, filename)
                try:
                    image = Image.open(image_path)
                    resized_image = image.resize((224, 224))
                    rotated_image = resized_image.rotate(180)
                    rotated_image.save(image_path)
                    print(f"Processed: {image_path}")
                except Exception as e:
                    print(f"Error processing {image_path}: {str(e)}")

if __name__ == "__main__":
    folder_path = r"C:\Users\MEY-SCH\Downloads\Sleep Monitor Project\dataset\Test Awais RGB_Light_on_Without Blanket\supine"  # Replace this with the actual folder path
    resize_and_rotate_images(folder_path)


Processed: C:\Users\MEY-SCH\Downloads\Sleep Monitor Project\dataset\Test Awais RGB_Light_on_Without Blanket\supine\ahsan (1).jpg
Processed: C:\Users\MEY-SCH\Downloads\Sleep Monitor Project\dataset\Test Awais RGB_Light_on_Without Blanket\supine\ahsan (10).jpg
Processed: C:\Users\MEY-SCH\Downloads\Sleep Monitor Project\dataset\Test Awais RGB_Light_on_Without Blanket\supine\ahsan (100).jpg
Processed: C:\Users\MEY-SCH\Downloads\Sleep Monitor Project\dataset\Test Awais RGB_Light_on_Without Blanket\supine\ahsan (101).jpg
Processed: C:\Users\MEY-SCH\Downloads\Sleep Monitor Project\dataset\Test Awais RGB_Light_on_Without Blanket\supine\ahsan (102).jpg
Processed: C:\Users\MEY-SCH\Downloads\Sleep Monitor Project\dataset\Test Awais RGB_Light_on_Without Blanket\supine\ahsan (103).jpg
Processed: C:\Users\MEY-SCH\Downloads\Sleep Monitor Project\dataset\Test Awais RGB_Light_on_Without Blanket\supine\ahsan (104).jpg
Processed: C:\Users\MEY-SCH\Downloads\Sleep Monitor Project\dataset\Test Awais RGB_Lig